In [ ]:
# 查看当前挂载的数据集目录
!ls /home/kesci/input/

In [ ]:
# 查看个人持久化工作区文件
!ls /home/kesci/work/

In [ ]:
# 查看当前kernerl下的package
!pip list --format=columns

In [2]:
import pandas as pd
pd.read_pickle("/home/kesci/work/first_zzp/test3.pickle")

In [1]:
# -*- coding:utf-8 -*-

import os
# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"  
# os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

import tensorflow as tf

from tensorflow.contrib import rnn

import tensorflow.contrib.layers as layers
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn import preprocessing
import tensorflow as tf
import gc
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from time import time
import numpy as np



class Settings(object):

    def __init__(self):


        self.model_name = 'wd_1_2_cnn_max'
        self.query_len = 20
        self.title_len = 20
        self.filter_sizes = [1,2, 3, 4, 6]
        self.n_filter = 350
        self.fc_hidden_size = 1024
        self.n_class = 1
        self.train_batch_size=1024
        self.vaild_batch_size=1024
        self.lr=0.0015
        self.decay_step=20000
        self.decay_rate=0.8
        self.rnn_hidden_units=170
        self.dense_feature_num=0
        self.ln_hidden_size = 128





class TEXTCNN(object):

    def __init__(self,  settings):


        self.model_name = settings.model_name
        self.query_len = settings.query_len
        self.title_len = settings.title_len
        self.filter_sizes = settings.filter_sizes
        self.n_filter = settings.n_filter
        self.n_filter_total = self.n_filter * len(self.filter_sizes)
        self.n_class = settings.n_class
        self.rnn_hidden_units=settings.rnn_hidden_units
        self.fc_hidden_size = settings.fc_hidden_size
        self._global_step = tf.Variable(0, trainable=False, name='Global_Step')
        self.update_emas = list()
        self.dense_feature_num = settings.dense_feature_num
        self.ln_hidden_size = settings.ln_hidden_size
        # placeholders
        self.is_training = tf.placeholder(tf.bool)
        self._keep_prob = tf.placeholder(tf.float32, [])
        self._batch_size = tf.placeholder(tf.int32, [])
        self.embedding_size=100
        
        self.dnn_all=[]

        with tf.name_scope('Inputs'):
            self.embedded_x_query=tf.placeholder(tf.float32,[None,self.query_len,self.embedding_size])#h r
            self.embedded_x_title=tf.placeholder(tf.float32,[None,self.title_len,self.embedding_size])# r
            self._y_inputs = tf.placeholder(tf.float32, [None, self.n_class], name='y_input')
            self.kv_features=tf.placeholder(shape=(None,self.dense_feature_num), dtype=tf.float32)


        with tf.name_scope("ln_output"):
            output_ln = self.ln_foward(self.kv_features)
            self.dnn_all.append(output_ln)

        with tf.name_scope("cnn_output"):
            output_title = self.cnn_inference(self.embedded_x_query, self.query_len ,'query_bn')
            output_title = tf.expand_dims(output_title, 0)

            output_content = self.cnn_inference(self.embedded_x_title, self.title_len ,'title_bn')
            output_content = tf.expand_dims(output_content, 0)
            output_cnn = tf.concat([output_title, output_content], axis=0)
            output_cnn = tf.reduce_max(output_cnn, axis=0)
            self.dnn_all.append(output_cnn)

        with tf.name_scope("rnn_output"):
            self.out1=self.BiRNN(self.embedded_x_query, self._keep_prob, "side1", self.embedding_size, self.query_len, self.rnn_hidden_units)
            self.out2=self.BiRNN(self.embedded_x_title, self._keep_prob, "side2", self.embedding_size, self.title_len, self.rnn_hidden_units)
            self.distance = tf.sqrt(tf.reduce_sum(tf.square(tf.subtract(self.out1,self.out2)),1,keep_dims=True))
            self.distance = tf.div(self.distance, tf.add(tf.sqrt(tf.reduce_sum(tf.square(self.out1),1,keep_dims=True)),tf.sqrt(tf.reduce_sum(tf.square(self.out2),1,keep_dims=True))))
            self.distance = tf.reshape(self.distance, [-1,1], name="distance")

        with tf.variable_scope('fc-bn-layer'):
            self.dnn_all = tf.concat(self.dnn_all, axis=1)
            dnn_bn = self.bn_layer_top(self.dnn_all,'dnn_bn_',self.is_training)

            fn_input_size = int(dnn_bn.shape[-1])
            W_fc = self.weight_variable([fn_input_size, self.fc_hidden_size], name='Weight_fc')
            b_fc = self.bias_variable([self.fc_hidden_size], name='fc_bias')
            fc_out = tf.nn.xw_plus_b(dnn_bn, W_fc, b_fc, name='y_pred')
            fc_relu = tf.nn.relu(fc_out, name="fc_relu")
            fc_bn = self.bn_layer_top(fc_relu,'fc_bn_',self.is_training)
            fc_drop = tf.nn.dropout(fc_bn, self.keep_prob)
            
            
        with tf.variable_scope('out_layer'):
            W_out = self.weight_variable([int(fc_drop.shape[-1]), self.n_class], name='Weight_out')
            b_out = self.bias_variable([self.n_class], name='bias_out')
            self._y_pred = tf.nn.xw_plus_b(fc_bn, W_out, b_out, name='y_pred')  # 每个类别的分数 scores
            self._y_pred = self._y_pred+self.distance

        with tf.name_scope('loss'):
            self._loss = tf.reduce_mean(
                tf.nn.sigmoid_cross_entropy_with_logits(logits=self._y_pred, labels=self._y_inputs))
            self._y_pred = tf.sigmoid(self._y_pred )

    @property
    def tst(self):
        return self._tst

    @property
    def keep_prob(self):
        return self._keep_prob

    @property
    def batch_size(self):
        return self._batch_size

    @property
    def global_step(self):
        return self._global_step

    @property
    def X1_inputs(self):
        return self._X1_inputs

    @property
    def X2_inputs(self):
        return self._X2_inputs

    @property
    def y_inputs(self):
        return self._y_inputs

    @property
    def y_pred(self):
        return self._y_pred

    @property
    def loss(self):
        return self._loss

    def weight_variable(self, shape, name):
        """Create a weight variable with appropriate initialization."""
        initial = tf.truncated_normal(shape, stddev=0.1)
        return tf.Variable(initial, name=name)

    def bias_variable(self, shape, name):
        """Create a bias variable with appropriate initialization."""
        initial = tf.constant(0.1, shape=shape)
        return tf.Variable(initial, name=name)


    def cnn_inference(self, X_inputs, n_step ,bn_name):
        """TextCNN 模型。
        Args:
            X_inputs: tensor.shape=(batch_size, n_step)
        Returns:
            title_outputs: tensor.shape=(batch_size, self.n_filter_total)
        """
        inputs = X_inputs
        inputs = tf.expand_dims(inputs, -1)
        pooled_outputs = list()
        for i, filter_size in enumerate(self.filter_sizes):
            with tf.variable_scope("conv-maxpool-%s" % filter_size):
                # Convolution Layer
                filter_shape = [filter_size, self.embedding_size, 1, self.n_filter]
                W_filter = self.weight_variable(shape=filter_shape, name='W_filter')
                beta = self.bias_variable(shape=[self.n_filter], name='beta_filter')
                conv = tf.nn.conv2d(inputs, W_filter, strides=[1, 1, 1, 1], padding="VALID", name="conv")
                conv = self.bn_layer_top(conv,bn_name,self.is_training)
                h = tf.nn.relu(conv, name="relu")
                # Maxpooling over the outputs
                pooled = tf.nn.max_pool(h, ksize=[1, n_step - filter_size + 1, 1, 1],
                                        strides=[1, 1, 1, 1], padding='VALID', name="pool")
                pooled_outputs.append(pooled)
        h_pool = tf.concat(pooled_outputs, 3)
        h_pool_flat = tf.reshape(h_pool, [-1, self.n_filter_total])
        return h_pool_flat  # shape = [batch_size, self.n_filter_total]

    def ln_foward(self,kv_features):
        kv_features=self.bn_layer(kv_features,'ln_bn_',self.is_training)
        input_size=int(kv_features.shape[-1])
        with tf.variable_scope('ln_model',reuse=False):
            W1 = tf.get_variable("W1" , [input_size , self.ln_hidden_size] , initializer = tf.contrib.layers.variance_scaling_initializer())
        Z1 = tf.matmul(kv_features , W1)
        A1 = tf.nn.relu(Z1)
        return A1



    def BiRNN(self, x, dropout, scope, embedding_size, sequence_length, hidden_units):
        n_hidden=hidden_units
        n_layers=3
        # Prepare data shape to match `static_rnn` function requirements
        x = tf.unstack(tf.transpose(x, perm=[1, 0, 2]))
        # Define lstm cells with tensorflow
        # Forward direction cell
        with tf.name_scope("fw"+scope),tf.variable_scope("fw"+scope):
            stacked_rnn_fw = []
            for _ in range(n_layers):
                fw_cell = tf.nn.rnn_cell.BasicLSTMCell(n_hidden, forget_bias=1.0, state_is_tuple=True)
                lstm_fw_cell = tf.contrib.rnn.DropoutWrapper(fw_cell,output_keep_prob=dropout)
                stacked_rnn_fw.append(lstm_fw_cell)
            lstm_fw_cell_m = tf.nn.rnn_cell.MultiRNNCell(cells=stacked_rnn_fw, state_is_tuple=True)

        with tf.name_scope("bw"+scope),tf.variable_scope("bw"+scope):
            stacked_rnn_bw = []
            for _ in range(n_layers):
                bw_cell = tf.nn.rnn_cell.BasicLSTMCell(n_hidden, forget_bias=1.0, state_is_tuple=True)
                lstm_bw_cell = tf.contrib.rnn.DropoutWrapper(bw_cell,output_keep_prob=dropout)
                stacked_rnn_bw.append(lstm_bw_cell)
            lstm_bw_cell_m = tf.nn.rnn_cell.MultiRNNCell(cells=stacked_rnn_bw, state_is_tuple=True)
        # Get lstm cell output

        with tf.name_scope("bw"+scope),tf.variable_scope("bw"+scope):
            outputs, _, _ = tf.nn.static_bidirectional_rnn(lstm_fw_cell_m, lstm_bw_cell_m, x, dtype=tf.float32)
        return outputs[-1]


    def bn_layer(self,x, scope, is_training, epsilon=0.001, decay=0.99, reuse=None):
        with tf.variable_scope(scope, reuse=reuse):
            shape = x.get_shape().as_list()
            gamma = tf.get_variable(scope+"_gamma", shape[-1], initializer=tf.constant_initializer(1.0), trainable=True)
            beta = tf.get_variable(scope+"_beta", shape[-1], initializer=tf.constant_initializer(0.0), trainable=True)
            moving_avg = tf.get_variable(scope+"_moving_mean", shape[-1], initializer=tf.constant_initializer(0.0), trainable=False)
            moving_var = tf.get_variable(scope+"_moving_variance", shape[-1], initializer=tf.constant_initializer(1.0), trainable=False)
            if is_training is not None:
                avg, var = tf.nn.moments(x, np.arange(len(shape)-1), keep_dims=True)
                avg=tf.reshape(avg, [avg.shape.as_list()[-1]])
                var=tf.reshape(var, [var.shape.as_list()[-1]])
                update_moving_avg=tf.assign(moving_avg, moving_avg*decay+avg*(1-decay))
                update_moving_var=tf.assign(moving_var, moving_var*decay+var*(1-decay))
                control_inputs = [update_moving_avg, update_moving_var]
            else:
                avg = moving_avg
                var = moving_var
                control_inputs = []
            with tf.control_dependencies(control_inputs):
                output = tf.nn.batch_normalization(x, avg, var, offset=beta, scale=gamma, variance_epsilon=epsilon)
        return output

    def bn_layer_top(self,x, scope, is_training, epsilon=0.001, decay=0.99):
        return tf.cond(
            is_training,
            lambda: self.bn_layer(x=x, scope=scope, epsilon=epsilon, decay=decay, is_training=True, reuse=None),
            lambda: self.bn_layer(x=x, scope=scope, epsilon=epsilon, decay=decay, is_training=False, reuse=True),
        )


    










def loader(data_store_path=''):############
    # all_data=pd.read_csv(data_store_path+"data_sets/train_feature_second_no_query_and_title_data.csv")
    # query_title=pd.read_csv(data_store_path+"data_sets/train_data.csv",usecols=['query','title'])

    print("开始加载数据，有ctr")
    # train_data=pd.read_pickle(data_store_path+"train_feature_first_no_query_and_title_data.pickle")[train_start_index:train_end_index].reset_index(drop=True)
    train_data=pd.read_pickle("/home/kesci/train_temp.pickle")
    query_title=pd.read_pickle("/home/kesci/query_title_temp.pickle")

    train_data=train_data.drop(['label'],axis=1)
    train_data=pd.concat([query_title,train_data],axis=1).reset_index(drop=True)
    data_split=StratifiedShuffleSplit(n_splits=2,test_size=0.001,random_state=1)
    
    train_index,vaild_index=data_split.split(train_data['label'],train_data['label']).__next__()
    train_data2=train_data.iloc[train_index]
    vaild_data2=train_data.iloc[vaild_index]
    print("加载数据完成")
    return train_data2,vaild_data2

def batch_iter(data_x,data_y, batch_size, num_epochs, shuffle=True):#这个就是产生batch的，可以直接用
    """
    Generates a batch iterator for a dataset.
    """
    # data_x=data_x.tolist()
    data=list(zip(data_x,data_y))
    data=np.array(data)
    data_size = len(data)
    num_batches_per_epoch = int((len(data)-1)/batch_size) + 1
    for epoch in range(num_epochs):
        # Shuffle the data at each epoch
        if shuffle:#如果为true 则代表允许随机取数据
            shuffle_indices = np.random.permutation(np.arange(data_size))
            shuffled_data = data[shuffle_indices]#随机取数据
        else:
            shuffled_data = data
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, data_size)
            yield shuffled_data[start_index:end_index]#而不用一个个加到列表里了。这是一个batch，嗯哼
            #取到的维度为（batchsize,2） y为(batchsize,1) 里面存的是序号

def get_w2v_array(word_list,max_len):
    array = np.zeros((max_len, 100))
    if len(word_list)<=max_len:
        for i in range(len(word_list)):
            if str(word_list[i]) in word_wv.vocab.keys():
                array[i][:] = word_wv[str(word_list[i])]
    else:
        for i in range(max_len):
            if str(word_list[i]) in word_wv.vocab.keys():
                array[i][:] = word_wv[str(word_list[i])]
    return array


settings = Settings()

train_batches,vaild_data=loader()

train_batches=np.array(train_batches)
train_batches_y=train_batches[:,2]
train_batches_x=np.delete(train_batches, 2, axis=1)######这里是除了label以外的维数


batches=batch_iter(train_batches_x,train_batches_y,settings.train_batch_size,4)

vaild_batches=np.array(vaild_data)
vaild_batches_y=vaild_batches[:,2]
vaild_batches_x=np.delete(vaild_batches, 2, axis=1)
vaild_data=batch_iter(vaild_batches_x,vaild_batches_y,settings.vaild_batch_size,1000)

print("train starting")
w2v_model = Word2Vec.load('w2v_model3/w2v_all_data_model.txt')
word_wv= w2v_model.wv


settings = Settings()

settings.dense_feature_num=train_batches_x.shape[1]-2###########自动设置数值型特征数量
model = TEXTCNN( settings)




update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(update_ops):
    learning_rate = tf.train.exponential_decay(settings.lr, model.global_step, settings.decay_step,
                                        settings.decay_rate, staircase=True)
    optimizer = tf.train.AdamOptimizer(learning_rate)
    grads_and_vars=optimizer.compute_gradients(model.loss)
    train_op=optimizer.apply_gradients(grads_and_vars,global_step=model.global_step)



config = tf.ConfigProto() 
config.gpu_options.allow_growth = True 
sess = tf.Session(config=config) #启动创建的模型
var_list = tf.trainable_variables()
g_list = tf.global_variables()
bn_moving_vars = [g for g in g_list if 'moving_mean' in g.name]
bn_moving_vars += [g for g in g_list if 'moving_variance' in g.name]
global_step_var=[g for g in g_list if 'Global_Step' in g.name]
var_list += bn_moving_vars
var_list += global_step_var
saver = tf.train.Saver(var_list)
sess.run(tf.global_variables_initializer())
saver.restore(sess, tf.train.latest_checkpoint('/home/kesci/work/tf_model_textcnn_siamese_ln3/'))

per_train_loss_avg_list=[]
per_vaild_loss_avg_list=[]
parts_time=[]
from time import time
for batch in batches:
    time1=time()
    batch_x,batch_y=zip(*batch)
    batch_y=np.array(batch_y)[:,None]
    batch_x=[i.tolist() for i in batch_x]
    batch_x=pd.DataFrame(batch_x)
    batch_x[[0,1]]=batch_x[[0,1]].applymap(lambda x:x.split(' '))
    batch_x=np.array(batch_x)
    #####[None,title_len,embedding_size]
    #np.apply_along_axis(lambda x:[5,5,5,5,5] if x[0]==2 else x,axis=2,arr=a)
    #query   title   title_count         title_ctr      label
    batch_x_query=np.apply_along_axis(lambda x:get_w2v_array(x[0],settings.query_len),axis=1,arr=batch_x)
    batch_x_title=np.apply_along_axis(lambda x:get_w2v_array(x[1],settings.title_len),axis=1,arr=batch_x)
    batch_x_ctr_fea=batch_x[:,2:]
    
    #     train_op=optimizer.apply_gradients(grads_and_vars,global_step=global_step)                              
    feed_dict = {model.embedded_x_query: batch_x_query,
                 model.embedded_x_title: batch_x_title,
                 model.y_inputs: batch_y,
                 model.batch_size: batch_x.shape[0], 
                 model.is_training: True, 
                 model.keep_prob: 0.8,
                 model.kv_features :batch_x_ctr_fea}

    _,step,loss_out=sess.run([train_op,model.global_step,model.loss],feed_dict)
    time2=time()
    parts_time.append(time2-time1)
    if step%10==0:
        print("step {},loss:{:g}".format(step,loss_out))              
    per_train_loss_avg_list.append(loss_out)
    
    
    if step%(int(50000000/1004))==0:
        print("batch ended")
    if step%500==0:
        time_single_part=sum(parts_time)
        ed_time=(50000000/settings.train_batch_size-step)/500*time_single_part
        print("距离训练完一整轮剩余时间：",ed_time/60," 分钟")
        parts_time=[]
        vaild_x,vaild_y=zip(*(vaild_data.__next__()))
        vaild_y=np.array(vaild_y)[:,None]
        vaild_x=[i.tolist() for i in vaild_x]
        vaild_x=pd.DataFrame(vaild_x)
        vaild_x[[0,1]]=vaild_x[[0,1]].applymap(lambda x:x.split(' '))
        vaild_x=np.array(vaild_x)
    
        vaild_x_query=np.apply_along_axis(lambda x:get_w2v_array(x[0],settings.query_len),axis=1,arr=vaild_x)
        vaild_x_title=np.apply_along_axis(lambda x:get_w2v_array(x[1],settings.title_len),axis=1,arr=vaild_x)

        vaild_x_ctr_fea=vaild_x[:,2:]

        feed_dict = {model.embedded_x_query: vaild_x_query,
                     model.embedded_x_title: vaild_x_title,
                     model.y_inputs: vaild_y,
                     model.batch_size: vaild_x.shape[0], 
                     model.is_training: False, 
                     model.keep_prob: 1,
                     model.kv_features :vaild_x_ctr_fea}
    
        step,loss_out=sess.run([model.global_step,model.loss],feed_dict)
        per_vaild_loss_avg_list.append(loss_out)
        per_train_loss_avg=sum(per_train_loss_avg_list)/len(per_train_loss_avg_list)
        print("per_train_loss_avg:",per_train_loss_avg,"**********")
        per_train_loss_avg_list=[]
        print("vaild_loss:",loss_out)
    if step%5000==0:
        per_vaild_loss_avg=sum(per_vaild_loss_avg_list)/len(per_vaild_loss_avg_list)
        per_vaild_loss_avg_list=[]
        print("per_vaild_loss_avg:",per_vaild_loss_avg,"************")

        var_list = tf.trainable_variables()
        g_list = tf.global_variables()
        bn_moving_vars = [g for g in g_list if 'moving_mean' in g.name]
        bn_moving_vars += [g for g in g_list if 'moving_variance' in g.name]
        global_step_var=[g for g in g_list if 'Global_Step' in g.name]
        var_list += global_step_var
        var_list += bn_moving_vars
        saver = tf.train.Saver(var_list=var_list)
        
        saver = saver.save(sess, "/home/kesci/work/tf_model_textcnn_siamese_ln3/rnn_fea", global_step=step)






In [1]:
for batch in batches:
    time1=time()
    batch_x,batch_y=zip(*batch)
    batch_y=np.array(batch_y)[:,None]
    batch_x=[i.tolist() for i in batch_x]
    batch_x=pd.DataFrame(batch_x)
    batch_x[[0,1]]=batch_x[[0,1]].applymap(lambda x:x.split(' '))
    batch_x=np.array(batch_x)
    #####[None,title_len,embedding_size]
    #np.apply_along_axis(lambda x:[5,5,5,5,5] if x[0]==2 else x,axis=2,arr=a)
    #query   title   title_count         title_ctr      label
    batch_x_query=np.apply_along_axis(lambda x:get_w2v_array(x[0],settings.query_len),axis=1,arr=batch_x)
    batch_x_title=np.apply_along_axis(lambda x:get_w2v_array(x[1],settings.title_len),axis=1,arr=batch_x)
    batch_x_ctr_fea=batch_x[:,2:]
    
    #     train_op=optimizer.apply_gradients(grads_and_vars,global_step=global_step)                              
    feed_dict = {model.embedded_x_query: batch_x_query,
                 model.embedded_x_title: batch_x_title,
                 model.y_inputs: batch_y,
                 model.batch_size: batch_x.shape[0], 
                 model.is_training: True, 
                 model.keep_prob: 0.8,
                 model.kv_features :batch_x_ctr_fea}

    _,step,loss_out=sess.run([train_op,model.global_step,model.loss],feed_dict)
    time2=time()
    parts_time.append(time2-time1)
    if step%10==0:
        print("step {},loss:{:g}".format(step,loss_out))              
    per_train_loss_avg_list.append(loss_out)
    
    
    if step%(int(50000000/1004))==0:
        print("batch ended")
    if step%500==0:
        time_single_part=sum(parts_time)
        ed_time=(50000000/settings.train_batch_size-step)/500*time_single_part
        print("距离训练完一整轮剩余时间：",ed_time/60," 分钟")
        parts_time=[]
        vaild_x,vaild_y=zip(*(vaild_data.__next__()))
        vaild_y=np.array(vaild_y)[:,None]
        vaild_x=[i.tolist() for i in vaild_x]
        vaild_x=pd.DataFrame(vaild_x)
        vaild_x[[0,1]]=vaild_x[[0,1]].applymap(lambda x:x.split(' '))
        vaild_x=np.array(vaild_x)
    
        vaild_x_query=np.apply_along_axis(lambda x:get_w2v_array(x[0],settings.query_len),axis=1,arr=vaild_x)
        vaild_x_title=np.apply_along_axis(lambda x:get_w2v_array(x[1],settings.title_len),axis=1,arr=vaild_x)

        vaild_x_ctr_fea=vaild_x[:,2:]

        feed_dict = {model.embedded_x_query: vaild_x_query,
                     model.embedded_x_title: vaild_x_title,
                     model.y_inputs: vaild_y,
                     model.batch_size: vaild_x.shape[0], 
                     model.is_training: False, 
                     model.keep_prob: 1,
                     model.kv_features :vaild_x_ctr_fea}
    
        step,loss_out=sess.run([model.global_step,model.loss],feed_dict)
        per_vaild_loss_avg_list.append(loss_out)
        per_train_loss_avg=sum(per_train_loss_avg_list)/len(per_train_loss_avg_list)
        print("per_train_loss_avg:",per_train_loss_avg,"**********")
        per_train_loss_avg_list=[]
        print("vaild_loss:",loss_out)
    if step%5000==0:
        per_vaild_loss_avg=sum(per_vaild_loss_avg_list)/len(per_vaild_loss_avg_list)
        per_vaild_loss_avg_list=[]
        print("per_vaild_loss_avg:",per_vaild_loss_avg,"************")

        var_list = tf.trainable_variables()
        g_list = tf.global_variables()
        bn_moving_vars = [g for g in g_list if 'moving_mean' in g.name]
        bn_moving_vars += [g for g in g_list if 'moving_variance' in g.name]
        global_step_var=[g for g in g_list if 'Global_Step' in g.name]
        var_list += global_step_var
        var_list += bn_moving_vars
        saver = tf.train.Saver(var_list=var_list)
        
        saver = saver.save(sess, "/home/kesci/work/tf_model_textcnn_siamese_ln2/rnn_fea", global_step=step)




In [1]:
# -*- coding:utf-8 -*-

import os
# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"  
# os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

import tensorflow as tf

from tensorflow.contrib import rnn

import tensorflow.contrib.layers as layers
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn import preprocessing
import tensorflow as tf
import gc
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from time import time
import numpy as np




class Settings(object):

    def __init__(self):


        self.model_name = 'wd_1_2_cnn_max'
        self.query_len = 20
        self.title_len = 20
        self.filter_sizes = [1,2, 3, 4, 6]
        self.n_filter = 256
        self.fc_hidden_size = 1024
        self.n_class = 1
        self.train_batch_size=1024
        self.vaild_batch_size=1024
        self.lr=0.0015
        self.decay_step=20000
        self.decay_rate=0.8
        self.rnn_hidden_units=120
        self.dense_feature_num=0
        self.ln_hidden_size = 512





class TEXTCNN(object):

    def __init__(self,  settings):


        self.model_name = settings.model_name
        self.query_len = settings.query_len
        self.title_len = settings.title_len
        self.filter_sizes = settings.filter_sizes
        self.n_filter = settings.n_filter
        self.n_filter_total = self.n_filter * len(self.filter_sizes)
        self.n_class = settings.n_class
        self.rnn_hidden_units=settings.rnn_hidden_units
        self.fc_hidden_size = settings.fc_hidden_size
        self._global_step = tf.Variable(0, trainable=False, name='Global_Step')
        self.update_emas = list()
        self.dense_feature_num = settings.dense_feature_num
        self.ln_hidden_size = settings.ln_hidden_size
        # placeholders
        self.is_training = tf.placeholder(tf.bool)
        self._keep_prob = tf.placeholder(tf.float32, [])
        self._batch_size = tf.placeholder(tf.int32, [])
        self.embedding_size=100
        
        self.dnn_all=[]

        with tf.name_scope('Inputs'):
            self.embedded_x_query=tf.placeholder(tf.float32,[None,self.query_len,self.embedding_size])#h r
            self.embedded_x_title=tf.placeholder(tf.float32,[None,self.title_len,self.embedding_size])# r
            self._y_inputs = tf.placeholder(tf.float32, [None, self.n_class], name='y_input')
            self.kv_features=tf.placeholder(shape=(None,self.dense_feature_num), dtype=tf.float32)


        with tf.name_scope("ln_output"):
            output_ln = self.ln_foward(self.kv_features)
            self.dnn_all.append(output_ln)

        with tf.name_scope("cnn_output"):
            output_title = self.cnn_inference(self.embedded_x_query, self.query_len ,'query_bn')
            output_title = tf.expand_dims(output_title, 0)

            output_content = self.cnn_inference(self.embedded_x_title, self.title_len ,'title_bn')
            output_content = tf.expand_dims(output_content, 0)
            output_cnn = tf.concat([output_title, output_content], axis=0)
            output_cnn = tf.reduce_max(output_cnn, axis=0)
            self.dnn_all.append(output_cnn)

        with tf.name_scope("rnn_output"):
            self.out1=self.BiRNN(self.embedded_x_query, self._keep_prob, "side1", self.embedding_size, self.query_len, self.rnn_hidden_units)
            self.out2=self.BiRNN(self.embedded_x_title, self._keep_prob, "side2", self.embedding_size, self.title_len, self.rnn_hidden_units)
            self.distance = tf.sqrt(tf.reduce_sum(tf.square(tf.subtract(self.out1,self.out2)),1,keep_dims=True))
            self.distance = tf.div(self.distance, tf.add(tf.sqrt(tf.reduce_sum(tf.square(self.out1),1,keep_dims=True)),tf.sqrt(tf.reduce_sum(tf.square(self.out2),1,keep_dims=True))))
            self.distance = tf.reshape(self.distance, [-1,1], name="distance")

        with tf.variable_scope('fc-bn-layer'):
            self.dnn_all = tf.concat(self.dnn_all, axis=1)
            dnn_bn = self.bn_layer_top(self.dnn_all,'dnn_bn_',self.is_training)

            fn_input_size = int(dnn_bn.shape[-1])
            W_fc = self.weight_variable([fn_input_size, self.fc_hidden_size], name='Weight_fc')
            b_fc = self.bias_variable([self.fc_hidden_size], name='fc_bias')
            fc_out = tf.nn.xw_plus_b(dnn_bn, W_fc, b_fc, name='y_pred')
            fc_relu = tf.nn.relu(fc_out, name="fc_relu")
            fc_bn = self.bn_layer_top(fc_relu,'fc_bn_',self.is_training)
            fc_drop = tf.nn.dropout(fc_bn, self.keep_prob)
            
            
        with tf.variable_scope('out_layer'):
            W_out = self.weight_variable([int(fc_drop.shape[-1]), self.n_class], name='Weight_out')
            b_out = self.bias_variable([self.n_class], name='bias_out')
            self._y_pred = tf.nn.xw_plus_b(fc_bn, W_out, b_out, name='y_pred')  # 每个类别的分数 scores
            self._y_pred = self._y_pred+self.distance

        with tf.name_scope('loss'):
            self._loss = tf.reduce_mean(
                tf.nn.sigmoid_cross_entropy_with_logits(logits=self._y_pred, labels=self._y_inputs))
            self._y_pred = tf.sigmoid(self._y_pred )

    @property
    def tst(self):
        return self._tst

    @property
    def keep_prob(self):
        return self._keep_prob

    @property
    def batch_size(self):
        return self._batch_size

    @property
    def global_step(self):
        return self._global_step

    @property
    def X1_inputs(self):
        return self._X1_inputs

    @property
    def X2_inputs(self):
        return self._X2_inputs

    @property
    def y_inputs(self):
        return self._y_inputs

    @property
    def y_pred(self):
        return self._y_pred

    @property
    def loss(self):
        return self._loss

    def weight_variable(self, shape, name):
        """Create a weight variable with appropriate initialization."""
        initial = tf.truncated_normal(shape, stddev=0.1)
        return tf.Variable(initial, name=name)

    def bias_variable(self, shape, name):
        """Create a bias variable with appropriate initialization."""
        initial = tf.constant(0.1, shape=shape)
        return tf.Variable(initial, name=name)


    def cnn_inference(self, X_inputs, n_step ,bn_name):
        """TextCNN 模型。
        Args:
            X_inputs: tensor.shape=(batch_size, n_step)
        Returns:
            title_outputs: tensor.shape=(batch_size, self.n_filter_total)
        """
        inputs = X_inputs
        inputs = tf.expand_dims(inputs, -1)
        pooled_outputs = list()
        for i, filter_size in enumerate(self.filter_sizes):
            with tf.variable_scope("conv-maxpool-%s" % filter_size):
                # Convolution Layer
                filter_shape = [filter_size, self.embedding_size, 1, self.n_filter]
                W_filter = self.weight_variable(shape=filter_shape, name='W_filter')
                beta = self.bias_variable(shape=[self.n_filter], name='beta_filter')
                conv = tf.nn.conv2d(inputs, W_filter, strides=[1, 1, 1, 1], padding="VALID", name="conv")
                conv = self.bn_layer_top(conv,bn_name,self.is_training)
                h = tf.nn.relu(conv, name="relu")
                # Maxpooling over the outputs
                pooled = tf.nn.max_pool(h, ksize=[1, n_step - filter_size + 1, 1, 1],
                                        strides=[1, 1, 1, 1], padding='VALID', name="pool")
                pooled_outputs.append(pooled)
        h_pool = tf.concat(pooled_outputs, 3)
        h_pool_flat = tf.reshape(h_pool, [-1, self.n_filter_total])
        return h_pool_flat  # shape = [batch_size, self.n_filter_total]

    def ln_foward(self,kv_features):
        kv_features=self.bn_layer(kv_features,'ln_bn_',self.is_training)
        input_size=int(kv_features.shape[-1])
        with tf.variable_scope('ln_model',reuse=False):
            W1 = tf.get_variable("W1" , [input_size , self.ln_hidden_size] , initializer = tf.contrib.layers.variance_scaling_initializer())
        Z1 = tf.matmul(kv_features , W1)
        A1 = tf.nn.relu(Z1)
        return A1



    def BiRNN(self, x, dropout, scope, embedding_size, sequence_length, hidden_units):
        n_hidden=hidden_units
        n_layers=3
        # Prepare data shape to match `static_rnn` function requirements
        x = tf.unstack(tf.transpose(x, perm=[1, 0, 2]))
        # Define lstm cells with tensorflow
        # Forward direction cell
        with tf.name_scope("fw"+scope),tf.variable_scope("fw"+scope):
            stacked_rnn_fw = []
            for _ in range(n_layers):
                fw_cell = tf.nn.rnn_cell.BasicLSTMCell(n_hidden, forget_bias=1.0, state_is_tuple=True)
                lstm_fw_cell = tf.contrib.rnn.DropoutWrapper(fw_cell,output_keep_prob=dropout)
                stacked_rnn_fw.append(lstm_fw_cell)
            lstm_fw_cell_m = tf.nn.rnn_cell.MultiRNNCell(cells=stacked_rnn_fw, state_is_tuple=True)

        with tf.name_scope("bw"+scope),tf.variable_scope("bw"+scope):
            stacked_rnn_bw = []
            for _ in range(n_layers):
                bw_cell = tf.nn.rnn_cell.BasicLSTMCell(n_hidden, forget_bias=1.0, state_is_tuple=True)
                lstm_bw_cell = tf.contrib.rnn.DropoutWrapper(bw_cell,output_keep_prob=dropout)
                stacked_rnn_bw.append(lstm_bw_cell)
            lstm_bw_cell_m = tf.nn.rnn_cell.MultiRNNCell(cells=stacked_rnn_bw, state_is_tuple=True)
        # Get lstm cell output

        with tf.name_scope("bw"+scope),tf.variable_scope("bw"+scope):
            outputs, _, _ = tf.nn.static_bidirectional_rnn(lstm_fw_cell_m, lstm_bw_cell_m, x, dtype=tf.float32)
        return outputs[-1]


    def bn_layer(self,x, scope, is_training, epsilon=0.001, decay=0.99, reuse=None):
        with tf.variable_scope(scope, reuse=reuse):
            shape = x.get_shape().as_list()
            gamma = tf.get_variable(scope+"_gamma", shape[-1], initializer=tf.constant_initializer(1.0), trainable=True)
            beta = tf.get_variable(scope+"_beta", shape[-1], initializer=tf.constant_initializer(0.0), trainable=True)
            moving_avg = tf.get_variable(scope+"_moving_mean", shape[-1], initializer=tf.constant_initializer(0.0), trainable=False)
            moving_var = tf.get_variable(scope+"_moving_variance", shape[-1], initializer=tf.constant_initializer(1.0), trainable=False)
            if is_training is not None:
                avg, var = tf.nn.moments(x, np.arange(len(shape)-1), keep_dims=True)
                avg=tf.reshape(avg, [avg.shape.as_list()[-1]])
                var=tf.reshape(var, [var.shape.as_list()[-1]])
                update_moving_avg=tf.assign(moving_avg, moving_avg*decay+avg*(1-decay))
                update_moving_var=tf.assign(moving_var, moving_var*decay+var*(1-decay))
                control_inputs = [update_moving_avg, update_moving_var]
            else:
                avg = moving_avg
                var = moving_var
                control_inputs = []
            with tf.control_dependencies(control_inputs):
                output = tf.nn.batch_normalization(x, avg, var, offset=beta, scale=gamma, variance_epsilon=epsilon)
        return output

    def bn_layer_top(self,x, scope, is_training, epsilon=0.001, decay=0.99):
        return tf.cond(
            is_training,
            lambda: self.bn_layer(x=x, scope=scope, epsilon=epsilon, decay=decay, is_training=True, reuse=None),
            lambda: self.bn_layer(x=x, scope=scope, epsilon=epsilon, decay=decay, is_training=False, reuse=True),
        )


    










def loader(data_store_path=''):############
    # all_data=pd.read_csv(data_store_path+"data_sets/train_feature_second_no_query_and_title_data.csv")
    # query_title=pd.read_csv(data_store_path+"data_sets/train_data.csv",usecols=['query','title'])

    print("开始加载数据，有ctr")
    # train_data=pd.read_pickle(data_store_path+"train_feature_first_no_query_and_title_data.pickle")[train_start_index:train_end_index].reset_index(drop=True)
    train_data=pd.read_pickle("/home/kesci/blending/5000w-6000wall_fea_norm.pickle")
    query_title=pd.read_pickle("/home/kesci/blending/train_data5000w-6000w.pickle")
    

    train_data=train_data.drop(['label'],axis=1)
    train_data=pd.concat([query_title,train_data],axis=1).reset_index(drop=True)

    print("加载数据完成")
    return train_data,1

def batch_iter(data_x,data_y, batch_size, num_epochs, shuffle=False):#这个就是产生batch的，可以直接用
    """
    Generates a batch iterator for a dataset.
    """
    # data_x=data_x.tolist()
    data=list(zip(data_x,data_y))
    data=np.array(data)
    data_size = len(data)
    num_batches_per_epoch = int((len(data)-1)/batch_size) + 1
    for epoch in range(num_epochs):
        # Shuffle the data at each epoch
        if shuffle:#如果为true 则代表允许随机取数据
            shuffle_indices = np.random.permutation(np.arange(data_size))
            shuffled_data = data[shuffle_indices]#随机取数据
        else:
            shuffled_data = data
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, data_size)
            yield shuffled_data[start_index:end_index]#而不用一个个加到列表里了。这是一个batch，嗯哼
            #取到的维度为（batchsize,2） y为(batchsize,1) 里面存的是序号

def get_w2v_array(word_list,max_len):
    array = np.zeros((max_len, 100))
    if len(word_list)<=max_len:
        for i in range(len(word_list)):
            if str(word_list[i]) in word_wv.vocab.keys():
                array[i][:] = word_wv[str(word_list[i])]
    else:
        for i in range(max_len):
            if str(word_list[i]) in word_wv.vocab.keys():
                array[i][:] = word_wv[str(word_list[i])]
    return array


settings = Settings()

train_batches,vaild_data=loader()

train_batches=np.array(train_batches)
train_batches_y=train_batches[:,2]
train_batches_x=np.delete(train_batches, 2, axis=1)######这里是除了label以外的维数


batches=batch_iter(train_batches_x,train_batches_y,settings.train_batch_size,1)



print("train starting")
w2v_model = Word2Vec.load('w2v_model3/w2v_all_data_model.txt')
word_wv= w2v_model.wv


settings = Settings()

settings.dense_feature_num=train_batches_x.shape[1]-2###########自动设置数值型特征数量
model = TEXTCNN( settings)




update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(update_ops):
    learning_rate = tf.train.exponential_decay(settings.lr, model.global_step, settings.decay_step,
                                        settings.decay_rate, staircase=True)
    optimizer = tf.train.AdamOptimizer(learning_rate)
    grads_and_vars=optimizer.compute_gradients(model.loss)
    train_op=optimizer.apply_gradients(grads_and_vars,global_step=model.global_step)

config = tf.ConfigProto() 
config.gpu_options.allow_growth = True 

sess = tf.Session(config=config) #启动创建的模型
var_list = tf.trainable_variables()
g_list = tf.global_variables()
bn_moving_vars = [g for g in g_list if 'moving_mean' in g.name]
bn_moving_vars += [g for g in g_list if 'moving_variance' in g.name]
global_step_var=[g for g in g_list if 'Global_Step' in g.name]
var_list += bn_moving_vars
var_list += global_step_var
saver = tf.train.Saver(var_list)
sess.run(tf.global_variables_initializer())
saver.restore(sess, '/home/kesci/work/tf_model_textcnn_siamese_ln2/rnn_fea-220000')


parts_time=[]
from time import time
result_list=[]
for step,batch in enumerate(batches):
    time1=time()
    batch_x,batch_y=zip(*batch)
    batch_y=np.array(batch_y)[:,None]
    batch_x=[i.tolist() for i in batch_x]
    batch_x=pd.DataFrame(batch_x)
    batch_x[[0,1]]=batch_x[[0,1]].applymap(lambda x:x.split(' '))
    batch_x=np.array(batch_x)
    #####[None,title_len,embedding_size]
    #np.apply_along_axis(lambda x:[5,5,5,5,5] if x[0]==2 else x,axis=2,arr=a)
    #query   title   title_count         title_ctr      label
    batch_x_query=np.apply_along_axis(lambda x:get_w2v_array(x[0],settings.query_len),axis=1,arr=batch_x)
    batch_x_title=np.apply_along_axis(lambda x:get_w2v_array(x[1],settings.title_len),axis=1,arr=batch_x)
    batch_x_ctr_fea=batch_x[:,2:]
    
    #     train_op=optimizer.apply_gradients(grads_and_vars,global_step=global_step)                              
    feed_dict = {model.embedded_x_query: batch_x_query,
                 model.embedded_x_title: batch_x_title,
                 model.batch_size: batch_x.shape[0], 
                 model.is_training: False, 
                 model.keep_prob: 1,
                 model.kv_features :batch_x_ctr_fea}

    result=sess.run([model._y_pred],feed_dict)
    result_list.append(result)
    time2=time()
    parts_time.append(time2-time1)
    if step%10==0:
        print(result[0][0])  
    if step%100==0:  
        time_single_part=sum(parts_time)
        ed_time=(20000000/settings.train_batch_size-step)/100*time_single_part
        print("距预测完一整轮剩余时间：",ed_time/60," 分钟")
        parts_time=[]
    
    

In [2]:
result_list2=[pd.DataFrame(i[0]) for i in result_list]
y_pred=pd.concat(result_list2).reset_index(drop=True)
y_pred

In [4]:
y_pred.to_pickle("/home/kesci/test_old_result/siamese_fea_no_ctr.pickle")

In [5]:
data_r_path='/home/kesci/input/bytedance/test_final_part1.csv'
cpu_num=16
start=0
all_data_num=20000000
names_out=['query_id','query','query_title_id','title']
usecols_out=['query_id','query_title_id']
has_head=False
if has_head==False:
    import multiprocessing
    from time import time
    import pandas as pd
    all_data=[]
    #test_final_part1.csv
    def data_read(start,single_data_num,data_real_path):
        data_out=pd.read_csv(data_real_path,header=None,names=names_out,usecols=usecols_out,skiprows=start,nrows=single_data_num)
        return data_out
    time1=time()
    pool = multiprocessing.Pool(processes=cpu_num)
    
    for epoch in range(int(cpu_num)):
        print(epoch)
        single_data_num=int(all_data_num/cpu_num)
        all_data.append(pool.apply_async(data_read, [start+single_data_num*epoch,single_data_num,data_r_path]))
    # single_data_num=20000000/16
    # all_data_num=20000000
    # for epoch in range(int(all_data_num/single_data_num)):
    #     all_data.append(pool.apply_async(data_read, [single_data_num*epoch,single_data_num,data_path+"test_final_part1.csv"]))
    pool.close()
    pool.join()
    time2=time()
    print(time2-time1)
    all_data_pro=[single.get() for single in all_data]
    time3=time()
    print(time3-time2)
    all_data_pro=pd.concat(all_data_pro)
    test_data_pred_need=all_data_pro.reset_index(drop=True)

result=pd.concat([test_data_pred_need,y_pred],axis=1)
result.to_csv("first_zzp/result/sub_siamese_fea_no_ctr.csv",header=None,index=None)#####合成三列做 最终的提交结果
result

In [7]:
!wget -O kesci_submit https://www.heywhale.com/kesci_submit&&chmod +x kesci_submit
!https_proxy="http://klab-external-proxy" ./kesci_submit -file /home/kesci/work/first_zzp/result/sub_t2.csv -token 02ada54c9760d3e1